In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

First of all, this is my first kaggle project. I'm very excited for this. I've been waiting for a long time to do this. Waiting for your suggestions about errors and omissions.

# All used libraries and modules

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

### Import dataset

In [ ]:
redwine = pd.read_csv('../input/red-wine-quality-cortez-et-al-2009/winequality-red.csv')

#always copy that

df = redwine.copy()

### Descriptive statistics and read data

In [ ]:
df.head()

In [ ]:
df.describe().T

In [ ]:
df.isnull().sum()

Cool! There is no null data here.

### Split data into training and test sets

I chose the quality classification as 1 and 0. As u guess 1 is good quality and 0 is bad. Stratify your sample will ensure your training set looks similar to your test set, making your evaluation metrics more reliable.

In [ ]:
df['quality'] = [1 if x>6.5 
                 else 0 
                 for x in df['quality']]
y = df["quality"]
X = df.drop("quality", axis=1)
X = pd.DataFrame(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.20, 
                                                    random_state=40,
                                                    stratify=y)

## K-NN

In [ ]:
knn_model = KNeighborsClassifier().fit(X_train, y_train)
y_pred = knn_model.predict(X_test)
accuracy_score(y_test, y_pred)


In [ ]:
knn_cv = GridSearchCV(KNeighborsClassifier(), {"n_neighbors": np.arange(1,50)}, cv=10)
knn_cv.fit(X_train, y_train)

In [ ]:
print("Best score is:" + str(knn_cv.best_score_),"and Best params is: " + str(knn_cv.best_params_))

In [ ]:
#OK. Let's use this for tuning.

knn = KNeighborsClassifier(2)
knn_tuned = knn.fit(X_train, y_train)

In [ ]:
knn_tuned.score(X_test, y_test)

### Random Forests

In [ ]:
df=redwine.copy()
df['quality'] = [1 if x>6.5 else 0 for x in df['quality']]
y = df["quality"]
X = df.drop("quality", axis=1)
X = pd.DataFrame(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.20, 
                                                    random_state=40)

In [ ]:
rf_model = RandomForestClassifier().fit(X_train, y_train)
y_pred = rf_model.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
rf_params = {"max_depth": [2,5,8,10],
            "max_features": [2,5,8],
            "n_estimators": [10,500,1000],
            "min_samples_split": [2,5,10]}

In [ ]:
rf_model = RandomForestClassifier()
rf_cv_model = GridSearchCV(rf_model, 
                           rf_params, 
                           cv = 10, 
                           n_jobs = -1, 
                           verbose = 2)
rf_cv_model.fit(X_train, y_train)

In [ ]:
rf_cv_model.best_params_

In [ ]:
#OK. Let's use this for tuning.
rf_tuned = RandomForestClassifier(max_depth = 10, 
                                  max_features = 2, 
                                  min_samples_split = 2, 
                                  n_estimators = 1000)

In [ ]:
rf_tuned.fit(X_train, y_train)
y_pred = rf_tuned.predict(X_test)
accuracy_score(y_test, y_pred)

### DataViz for variable importance level

In [ ]:
Importance = pd.DataFrame({"Importance": rf_tuned.feature_importances_*100},
                         index = X_train.columns)
Importance.sort_values(by = "Importance", 
                       axis = 0, 
                       ascending = True).plot(kind ="barh", color = "g")

plt.xlabel("Variable Importance")